<a href="https://colab.research.google.com/github/ronyates47/Gedcom-Utils/blob/main/2023_0912_2115_multi_cell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
!pip install pandas
!pip install python-gedcom

In [131]:
# total script


#Cell 1: Import Libraries
import glob
from gedcom.element.individual import IndividualElement
from gedcom.parser import Parser
import pandas as pd
import os

#Cell 2:Initialize Global Variables

# Initialize an empty dictionary to store dnalines
dnaline_dict = {}

# Reset the global variables
find_parents_new_counter = 0
last_pair_counter = 0
visited_pairs = set()

# Initialize the last_prime_surname variable
last_prime_surname = 'Yates'

# Cell 3: Define Gedcom Class

import os
import pandas as pd

# Adding filter_individuals method to the Gedcom class
# Gedcom Class Definition
class Gedcom:
    def __init__(self, file_name):
        self.file_name = file_name
        self.gedcom_datasets = []
        self.filter_pool = []
        self.parse_gedcom()
        self.filter_individuals()

    def parse_gedcom(self):
        # Assuming this is where your existing parse_gedcom code is
        with open(self.file_name, 'r', encoding='utf-8-sig') as f:
            gedcom_lines = f.readlines()

        current_dataset = None
        for line in gedcom_lines:
            parts = line.strip().split(' ', 2)
            level = int(parts[0])
            tag = parts[1]
            value = parts[2] if len(parts) > 2 else None

            if level == 0 and tag.startswith('@') and tag.endswith('@') and value == 'INDI':
                current_dataset = GedcomDataset(tag.strip('@'))
                self.gedcom_datasets.append(current_dataset)
            elif current_dataset is not None:
                if level == 1 and tag in ['NAME', 'FAMC']:
                    current_key = tag
                    current_dataset.add_extractable_detail(current_key, value)
                elif level == 2 and tag == 'NPFX':
                    current_dataset.add_extractable_detail(tag, value)
    def filter_individuals(self):
        if os.path.exists('/content/shortged.xlsx'):
            df = pd.read_excel('/content/shortged.xlsx')
            allowed_individual_ids = df.iloc[:, 0].dropna().str.strip().tolist()
            print("Allowed IDs from Excel file:", allowed_individual_ids)

            not_allowed_ids = []
            new_gedcom_datasets = []

            for dataset in self.gedcom_datasets:
                stripped_id = dataset.individual_id.strip()
                if stripped_id in allowed_individual_ids:
                    new_gedcom_datasets.append(dataset)
                else:
                    not_allowed_ids.append(stripped_id)

        self.gedcom_datasets = new_gedcom_datasets

        print('Records Moved into short_pool:', len(self.gedcom_datasets))
        print('IDs Not Allowed:', not_allowed_ids)

        for dataset in self.gedcom_datasets:
            print(type(dataset))

            if dataset.get_extractable_NPFX():
                self.filter_pool.append(dataset)

        print("Before_2023-09-12-1750_clearing ancestral_line:")
        for dataset in self.gedcom_datasets:
            print(dataset.ancestral_line)

        for dataset in self.gedcom_datasets:
            dataset.ancestral_line = []

        print("After clearing ancestral_line:")
        for dataset in self.gedcom_datasets:
            print(dataset.ancestral_line)

class GedcomDataset:
    def __init__(self, individual_id):
        self.individual_id = individual_id
        self.extractable_detail = {}
        self.anchor_gen1 = None
        self.ancestral_line = []

    def add_extractable_detail(self, key, value):
        self.extractable_detail[key] = value

    def get_gen_person(self):
        return self.individual_id

    def get_extractable_cm(self):
        return self.extractable_detail.get('cM', '')  # Assuming 'cM' is the key you use for centiMorgans

    def get_extractable_sort(self):
        return self.extractable_detail.get('Sort', '')  # Assuming 'Sort' is the key you use for sorting

    def get_extractable_NPFX(self):
        return self.extractable_detail.get('NPFX', '')

    def get_anchor_gen1(self):
        return self.anchor_gen1

def process_individual(individual_id, gedcom_instance):
    global generation_table
    generation_table = pd.DataFrame(columns=['Generation', 'Parent Pair'])
    global visited_pairs
    visited_pairs = set()

    find_parents(individual_id, 1, gedcom_instance.gedcom_datasets)
    anchor_gen1 = None  # Initialize as None to ensure it gets a value later

    # Concatenate the value of the anchor_gen1 attribute to the beginning of the generation table
    for dataset in gedcom_instance.filter_pool:
        if dataset.get_gen_person() == individual_id:
            anchor_gen1 = dataset.get_anchor_gen1()
            generation_table.loc[0] = [1, anchor_gen1]
            break

    generation_table = generation_table.sort_values('Generation', ascending=False).reset_index(drop=True)

    individual_data = {}
    for dataset in gedcom_instance.filter_pool:
        if dataset.get_gen_person() == individual_id:
            individual_data['cM'] = dataset.get_extractable_cm()
            individual_data['Sort'] = dataset.get_extractable_sort()
            break

    individual_data['Parent Pairs A10'] = '|'.join([f"{row['Parent Pair']}" for index, row in generation_table[:10].iterrows()])

    # Add the anchor_gen1 to individual_data
    individual_data['anchor_gen1'] = anchor_gen1  # Add this line

    return individual_data  # Now we only have one return statement that returns all the data

def input_prime_surname(last_prime_surname=None):
    global surname  # Declare surname as a global variable
    if last_prime_surname:
        last_name = input(f"Enter prime_surname (default: {last_prime_surname}): ")
        if not last_name:
            last_name = last_prime_surname
    else:
        last_name = input("Enter prime_surname: ")
    surname = last_name  # Assign the value of last_name to the global surname variable
    return last_name

# Call the function to let the user input prime_surname
prime_surname = input_prime_surname(last_prime_surname)

# Store the value of prime_surname for later use
last_prime_surname = prime_surname
#prime_surname = input_prime_surname(last_prime_surname)


gedcom_file_path = select_gedcom_file()

if gedcom_file_path:
    # Use the selected GEDCOM file path to create an instance of the Gedcom class
    gedcom_instance = Gedcom(gedcom_file_path)
    gedcom_instance.parse_gedcom()

# Initialize the list of individuals
    individuals = []

    # Iterate over the filter_pool list and add each individual's last name and ID to the individuals list
    for dataset in gedcom_instance.filter_pool:
        individual_id = dataset.get_gen_person()
        last_name = dataset.get_anchor_gen1()
        individuals.append((last_name, individual_id))

    print(f'Records Moved into Samaller Pile {len(individuals)}')

    # Read the GEDCOM file and split it into individual and family records
    with open(gedcom_file_path, 'r') as file:
        data = file.read()
    data = data.split('\n0 ')
    records = {extract_id(record): record for record in data}

# cell 12

parent_pairs = []
children = {}
last_pair_counter = 0
last_pair_counter = 0
find_parents_new_counter = 0

parent_pairs = []
children = {}
ancestral_lines = {}

def find_parents_new(individual_id, generation, records, parent_pairs, children, ancestral_lines, current_line=None):
    if current_line is None:
        current_line = []

    if individual_id not in records:
        return
    record = records[individual_id]
    famc_start = record.find('1 FAMC @') + 8
    famc_end = record.find('@', famc_start)
    famc_id = record[famc_start:famc_end]
    if famc_id not in records:
        return

    fam_record = records[famc_id]
    wife_start = fam_record.find('1 WIFE @') + 8
    wife_end = fam_record.find('@', wife_start)
    mother_id = fam_record[wife_start:wife_end]

    husb_start = fam_record.find('1 HUSB @') + 8
    husb_end = fam_record.find('@', husb_start)
    father_id = fam_record[husb_start:husb_end]

    if mother_id and mother_id in records:
        mother_record = records[mother_id]
        mother_name = extract_name(mother_record)
        if mother_name not in children:
            children[mother_name] = []
        children[mother_name].append(individual_id)
    else:
        mother_name = None

    if father_id and father_id in records:
        father_record = records[father_id]
        father_name = extract_name(father_record)
        if father_name not in children:
            children[father_name] = []
        children[father_name].append(individual_id)
    else:
        father_name = None

    if mother_name is not None and father_name is not None:
        parent_pair = father_name + "&" + mother_name
        current_line.append((generation, parent_pair))
        if parent_pair not in visited_pairs:
            visited_pairs.add(parent_pair)
            if has_both_parents(records, mother_id, father_id):
                parent_pairs.append((generation, parent_pair))
                ancestral_lines[parent_pair] = list(current_line)

    if mother_id:
        find_parents_new(mother_id, generation + 1, records, parent_pairs, children, ancestral_lines, current_line)

    if father_id:
        find_parents_new(father_id, generation + 1, records, parent_pairs, children, ancestral_lines, current_line)



    if current_line:  # Check if the list is not empty
        current_line.pop()

# cell 13

ancestral_lines = {}
find_parents_new(individual_id, 1, records, parent_pairs, children, ancestral_lines)

for last_pair, ancestral_line in ancestral_lines.items():
#    print(f'Head of Branch line (100 each surname) {last_pair}:')
    for generation, parent_pair in ancestral_line:
#        print(f'  Descendants of this branch line (1 each surname) {generation}: {parent_pair}')

# Count the total number of parent pairs found
        parent_pair_count = len(parent_pairs)
#        print(f'{parent_pair_count} parent pairs were found')

# NEW LINE: Count the total number of last pairs found
last_pair_count = len(ancestral_lines)  # Count the number of keys in the dictionary
print(f'{last_pair_count} (20230908-1438) last pairs were found')  # Print the count

# NEW: Count the total number of last pairs found
last_pair_count = len(ancestral_lines)
print(f'{last_pair_count} last pairs were found')


def process_individual_new(individual_id, gedcom_instance):
    global generation_table
    generation_table = pd.DataFrame(columns=['Generation', 'Parent Pair'])
    global visited_pairs
    visited_pairs = set()

    parent_pairs = []
    children = {}
    ancestral_lines = {}

    # Changed records to gedcom_instance.gedcom_datasets
    find_parents_new(individual_id, 1, gedcom_instance.gedcom_datasets, parent_pairs, children, ancestral_lines)

    anchor_gen1 = None  # Initialize as None to ensure it gets a value later

    # Fetch anchor_gen1 from GedcomDataset
    for dataset in gedcom_instance.filter_pool:
        if dataset.get_gen_person() == individual_id:
            anchor_gen1 = dataset.get_anchor_gen1()
            break

    # Add anchor_gen1 to the beginning of each ancestral line
    for last_pair, ancestral_line in ancestral_lines.items():
        if anchor_gen1 is not None:  # Check if anchor_gen1 has been assigned a value
            ancestral_line.insert(0, (1, anchor_gen1))

    individual_data = {}
    individual_data['Last Pairs'] = ancestral_lines
    individual_data['anchor_gen1'] = anchor_gen1  # Add anchor_gen1 to individual_data

    return individual_data


individual_data = process_individual_new(individual_id, gedcom_instance)
last_pairs = individual_data['Last Pairs']



def trace_children(individual_id, parent_pairs, children, visited=None):
    # Create a list to store the children of the individual
    individual_children = []

    # Create a set to keep track of visited individuals
    if visited is None:
        visited = set()

    # Check if the individual has already been visited
    if individual_id in visited:
        # If the individual has already been visited, stop the recursion and return an empty list
        return individual_children

    # Add the individual to the set of visited individuals
    visited.add(individual_id)

    # Find the name of the individual
    individual_name = None
    for pair in parent_pairs:
        if "lastpair" in pair[1]:
            names = pair[1].split('&')
            if individual_id in children[names[0]]:
                individual_name = names[0]
            elif individual_id in children[names[1]]:
                individual_name = names[1]
            break

    # Check if the individual has any children
    if individual_name in children:
        # Add the children of the individual to the list
        individual_children.extend(children[individual_name])

        # Recursively find the children of the individual's children
        for child_id in children[individual_name]:
            child_children = trace_children(child_id, parent_pairs, children, visited)
            individual_children.extend(child_children)

    return individual_children

descendants = trace_children(individual_id, parent_pairs, children)
#print(f'The ancestors of individual {individual_id} are: {descendants}')

import pandas as pd  # If you haven't already imported it

# user enters the surname=target surname
target_surname = surname

# Initialize a counter for the sub-branches
branch_sub_counter = 0

# Initialize your dictionary here
branch_surname_count = {}
formatted_lines_list = []  # This list will hold all your formatted ancestral lines for writing to Excel

# Initialize variables to keep track of the highest worth and its corresponding branch line number
highest_worth = 0
highest_worth_reference = None

# Initialize a dictionary to store the formatted_ancestral_line for each unique_reference
formatted_lines = {}

# Loop through each ancestral line and print the details
for last_pair, ancestral_line in ancestral_lines.items():

    # Increment the branch sub counter
    branch_sub_counter += 1

    # Create a unique reference for this branch
    unique_reference = f"branch line number: {branch_sub_counter}"

    # Initialize counters for this specific branch
    head_count = 0
    descendant_count = 0
    generations = len(ancestral_line)  # Count the number of generations for the tiebreaker

    if target_surname in last_pair:
        head_count += 1

    # Reverse the order of the ancestral line so that it goes from oldest to newest
    reversed_ancestral_line = reversed(ancestral_line)

    # Create the formatted ancestral line string
    formatted_ancestral_line = "|".join([pair for gen, pair in reversed_ancestral_line])

    # Add the formatted line to the list
    formatted_lines_list.append(formatted_ancestral_line)

    # Store this formatted_ancestral_line in our dictionary
    formatted_lines[unique_reference] = formatted_ancestral_line

    # Now use last_pair_in_line as the head of the branch line

    if target_surname in last_pair:  # I've assumed you meant the last pair in the ancestral line
            descendant_count += 1  # Increment the counter for 'Descendant'

    # Calculate the total worth for this branch
    total_worth = (head_count * 100) + descendant_count

    # Tie-breaking logic
    if total_worth > highest_worth or (total_worth == highest_worth and generations > branch_surname_count.get(highest_worth_reference, {}).get('Generations', 0)):
        highest_worth = total_worth
        highest_worth_reference = unique_reference

    # Store the counts in our dictionary
    branch_surname_count[unique_reference] = {'Head': head_count, 'Descendant': descendant_count, 'Total Worth': total_worth, 'Generations': generations}

# After the loop, highest_worth_reference holds the unique_reference of the branch line with the highest worth
# Assign the formatted_ancestral_line of the highest worth branch to dnaline

dnaline = formatted_lines[highest_worth_reference]
dnaline_dict[individual_id] = dnaline  # <-- This line saves dnaline to dnaline_dict for the current individual_id

# Once your loop is done, use pandas to write the list to an Excel file
#df = pd.DataFrame(formatted_lines_list, columns=['Formatted Ancestral Lines'])
#df.to_excel('Formatted_Ancestral_Lines.xlsx', index=False)

#cell 14 PRINTS TRADITIONAL ANCESTRAL LINES

import pandas as pd  # Make sure to import pandas

# THIS PREPARES THE OUTPUT USING THE ORIGINAL FULL ANCESTRAL LINE
#combined_df_rows = []
#for name, individual_id in individuals:
#    individual_data = process_individual(individual_id, gedcom_instance)
#    cm = individual_data['cM']
#    sort = individual_data['Sort']
#    parent_pairs_a10 = individual_data['Parent Pairs A10']
#    most_distant_ancestor = generation_table.iloc[0]['Parent Pair']  # Assuming generation_table exists
#    combined_df_rows.append([individual_id, name, sort, cm, most_distant_ancestor, parent_pairs_a10])


# Print column names for debugging
#    print("Current columns in combined_df:", combined_df.columns)

# Create DataFrame
#combined_df = pd.DataFrame(combined_df_rows, columns=['ID#', 'Name', 'Match to', 'cM', 'Most Distant Ancestor', 'Ancestral Line A10'])

# Create hotlinks
#def create_hotlink(row):
#    url_base = "https://yates.one-name.net/tng/verticalchart.php?personID="
#    person_id = row['ID#']
#    hotlink = f'<a href="{url_base}{person_id}&tree=tree1&parentset=0&display=vertical&generations=8" target="_blank">{person_id}</a>'
#    return hotlink

# Apply hotlink function
#combined_df['LUN#'] = combined_df.apply(lambda row: create_hotlink(row), axis=1)

# Adjust index
#combined_df.index = combined_df.index + 1

# Print DataFrame
#print(combined_df)

# Export to Excel
#combined_df.to_excel('/content/output.xlsx', index=False)

# cell 15  PRINTS DNA ANCESTRAL LINES

dnaline = formatted_lines[highest_worth_reference]
print(f"The branch with the highest worth is {highest_worth_reference}, and its DNA line is {dnaline}")

# This should be placed after you define 'dnaline = formatted_lines[highest_worth_reference]'
most_distant_ancestor = dnaline.split("|")[0]

# Then, in the combined_df_rows.append line
combined_df_rows.append([individual_id, name, sort, cm, most_distant_ancestor, parent_pairs_a10])

import pandas as pd  # Make sure to import pandas

# ... some code that should populate dnaline_dict ...

# Print the contents of dnaline_dict to debug
if not dnaline_dict:
    print("HEY! THERE IS NOTHING IN dnaline_dict!")
else:
    print("Contents of dnaline_dict:", dnaline_dict)

# ... code for creating or modifying combined_df ...



# THIS PREPARES THE OUTPUT USING THE SPECIAL DNALINE ANCESTRAL LINE
combined_df_rows = []
for name, individual_id in individuals:
    individual_data = process_individual(individual_id, gedcom_instance)

    cm = individual_data['cM']  # cM value is already in individual_data
    sort = individual_data['Sort']  # Sort value is already in individual_data
    parent_pairs_a10 = individual_data['Parent Pairs A10']
    anchor_gen1 = individual_data['anchor_gen1']  # Assuming you've stored it in individual_data
    most_distant_ancestor = generation_table.iloc[0]['Parent Pair']  # Assuming generation_table exists

# Print column names for debugging
    print("Current columns in combined_df:", combined_df.columns)

    combined_df_rows.append([individual_id, anchor_gen1, sort, cm, most_distant_ancestor, {dnaline}])  # Replaced {dnaline} with dnaline

# Create DataFrame
combined_df = pd.DataFrame(combined_df_rows, columns=['ID#', 'Name', 'Match to', 'cM', 'most_distant_ancestor', 'dnaline'])


# Create hotlinks
def create_hotlink(row):
    url_base = "https://yates.one-name.net/tng/verticalchart.php?personID="
    person_id = row['ID#']
    hotlink = f'<a href="{url_base}{person_id}&tree=tree1&parentset=0&display=vertical&generations=8" target="_blank">{person_id}</a>'
    return hotlink

# Apply hotlink function
combined_df['LUN#'] = combined_df.apply(lambda row: create_hotlink(row), axis=1)



# Adjust index
combined_df.index = combined_df.index + 1

# Print DataFrame
print(combined_df)

# Export to Excel
combined_df.to_excel('/content/output.xlsx', index=False)


Enter prime_surname (default: Yates): 
List of GEDCOM files:
1. yates-one-name-study.ged
Allowed IDs from Excel file: ['I42478', 'I45104', 'I46593', 'I50522', 'I26925']
Records Moved into short_pool: 5
IDs Not Allowed: ['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'I16', 'I17', 'I18', 'I19', 'I20', 'I21', 'I22', 'I23', 'I24', 'I25', 'I26', 'I27', 'I28', 'I29', 'I30', 'I31', 'I32', 'I33', 'I34', 'I35', 'I36', 'I37', 'I38', 'I39', 'I40', 'I41', 'I42', 'I43', 'I44', 'I45', 'I46', 'I47', 'I48', 'I49', 'I50', 'I51', 'I52', 'I53', 'I54', 'I55', 'I56', 'I57', 'I58', 'I59', 'I60', 'I61', 'I62', 'I63', 'I64', 'I65', 'I66', 'I67', 'I68', 'I69', 'I70', 'I71', 'I72', 'I73', 'I74', 'I75', 'I76', 'I77', 'I78', 'I79', 'I80', 'I81', 'I82', 'I83', 'I84', 'I85', 'I86', 'I87', 'I88', 'I89', 'I90', 'I91', 'I92', 'I93', 'I94', 'I95', 'I96', 'I97', 'I98', 'I99', 'I100', 'I101', 'I102', 'I103', 'I104', 'I105', 'I106', 'I107', 'I108', 'I109', 'I110', 'I111', 